# Random Forest Classification

In [1]:
# Import sections
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import sklearn.preprocessing as LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve)
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize

### II. Load Dataset

In [113]:
# =============================================================================
# Function: load_data
# =============================================================================
def load_data(filepath):
    """
    Load the KDD Cup 1999 dataset from the specified filepath.

    Parameters:
    - filepath (str): Path to the CSV file containing the dataset.

    Returns:
    - DataFrame: Loaded pandas DataFrame with appropriate column names.
    """
    try:
        df = pd.read_csv(filepath, header=0)
        return df
    except Exception as e:
        print(f"Failed to load dataset: {e}")
        return None
    
# Load the dataset
df = load_data('../../Data/kddcup.data_corrected.csv')
print(df)

        duration protocol_type  service flag  src_bytes  dst_bytes  land  \
0              0           udp  private   SF        105        146     0   
1              0           udp  private   SF        105        146     0   
2              0           udp  private   SF        105        146     0   
3              0           udp  private   SF        105        146     0   
4              0           udp  private   SF        105        146     0   
...          ...           ...      ...  ...        ...        ...   ...   
311024         0           udp  private   SF        105        147     0   
311025         0           udp  private   SF        105        147     0   
311026         0           udp  private   SF        105        147     0   
311027         0           udp  private   SF        105        147     0   
311028         0           udp  private   SF        105        147     0   

        wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0                    0 

### III. Preprocess Data

In [114]:
# =============================================================================
# Function: preprocess_data
# =============================================================================
def preprocess_data(df):
    """
    Clean and preprocess the dataset:
      - Remove duplicates.
      - Encode categorical variables using one-hot encoding.
      - Encode the target variable ('label') into binary format (0 for 'normal.', 1 for attacks).
      - Scale numerical features using StandardScaler.
      - Save the cleaned dataset for submission.

    Parameters:
    - df (DataFrame): The raw dataset.

    Returns:
    - X (DataFrame): Preprocessed features.
    - y (Series): Encoded target variable.
    - df_cleaned (DataFrame): Combined cleaned dataset with features and label.
    """
    # Loại bỏ các dòng trùng lặp
    df = df.drop_duplicates()

    # One-hot encode cho các biến phân loại: 'protocol_type', 'service', và 'flag'
    categorical_cols = ['protocol_type', 'service', 'flag']
    df = pd.get_dummies(df, columns=categorical_cols)

    # Mã hóa biến mục tiêu: gán 'normal.' thành 0, và các loại tấn công khác thành 1
    df['label'] = df['label'].apply(lambda x: 0 if x.strip() == 'normal.' else 1)

    # Tách các đặc trưng và biến mục tiêu
    X = df.drop('label', axis=1)
    y = df['label']

    # Chuẩn hóa các đặc trưng số
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X = pd.DataFrame(X_scaled, columns=X.columns)

    # Lưu dataset đã được làm sạch
    df_cleaned = X.copy()
    df_cleaned['label'] = y.values
    output_file = 'cleaned_kddcup_data.csv'
    df_cleaned.to_csv(output_file, index=False)
    print(f"Cleaned dataset saved as '{output_file}'.")

    return X, y, df_cleaned

df = load_data('../../Data/kddcup.data_corrected.csv')
preprocess_data(df)

Cleaned dataset saved as 'cleaned_kddcup_data.csv'.


(       duration  src_bytes  dst_bytes      land  wrong_fragment    urgent  \
 0     -0.086794  -0.013008  -0.084545 -0.010792       -0.032676 -0.010508   
 1     -0.086794  -0.013008  -0.084545 -0.010792       -0.032676 -0.010508   
 2     -0.086794  -0.013008  -0.084545 -0.010792       -0.032676 -0.010508   
 3     -0.086794  -0.013008  -0.084545 -0.010792       -0.032676 -0.010508   
 4     -0.086794  -0.013305  -0.089075 -0.010792       -0.032676 -0.010508   
 ...         ...        ...        ...       ...             ...       ...   
 77286 -0.086794  -0.013301  -0.089075 -0.010792       -0.032676 -0.010508   
 77287 -0.086794  -0.013301  -0.089075 -0.010792       -0.032676 -0.010508   
 77288 -0.086794  -0.013008  -0.085817 -0.010792       -0.032676 -0.010508   
 77289 -0.086794  -0.013301  -0.089075 -0.010792       -0.032676 -0.010508   
 77290 -0.086794  -0.013301  -0.089075 -0.010792       -0.032676 -0.010508   
 
             hot  num_failed_logins  logged_in  num_compromise

### IV. Vẽ biểu đồ classification report

In [115]:
def plot_confusion_and_classification(y_true, y_pred, labels=None):
    """
    Vẽ kết hợp Heatmap Confusion Matrix và biểu đồ Classification Report (gồm cả Accuracy).

    Parameters:
    - y_true: Ground truth labels
    - y_pred: Predicted labels
    - labels: Danh sách nhãn lớp (tuỳ chọn)
    """
    # --- Confusion Matrix ---
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    label_names = labels if labels else sorted(list(set(y_true)))
    cm_df = pd.DataFrame(cm, index=label_names, columns=label_names)

    # --- Classification Report ---
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    report_df = pd.DataFrame(report).T[['precision', 'recall', 'f1-score']]

    # Thêm Accuracy vào report (cho tất cả lớp) – đưa lên hàng cuối
    accuracy = accuracy_score(y_true, y_pred)
    report_df.loc['accuracy'] = [accuracy, accuracy, accuracy]

    # Lấy các lớp + 'accuracy'
    report_df_filtered = report_df.loc[list(report_df.index)]

    # --- Plot ---
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))

    # 1. Heatmap Confusion Matrix
    sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axs[0])
    axs[0].set_title("Confusion Matrix")
    axs[0].set_xlabel("Predicted")
    axs[0].set_ylabel("Actual")

    # 2. Classification Report Bar Chart + Accuracy
    report_df_filtered.plot(kind='bar', ax=axs[1], color=['skyblue', 'orange', 'green'])
    axs[1].set_title("Classification Report (including Accuracy)")
    axs[1].set_ylim(0, 1)
    axs[1].set_ylabel("Score")
    axs[1].grid(axis='y')
    axs[1].legend(loc='lower right')

    plt.tight_layout()
    plt.show()


### V. Train and evaluate

In [116]:
# =============================================================================
# Function: train_and_evaluate
# =============================================================================
def train_and_evaluate(X, y):
    """
    Train a Random Forest classifier and evaluate its performance using several metrics.
    Also, plot the ROC curve and feature importances.

    Parameters:
    - X (DataFrame): Preprocessed features.
    - y (Series): Encoded target variable.

    Returns:
    - model: The trained RandomForestClassifier.
    """
    # 1. Chia dữ liệu
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    # 2. Huấn luyện mô hình
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # 3. Dự đoán
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)

    plot_confusion_and_classification(y_test, y_pred)
    
    # 4. Lấy xác suất lớp dương (nếu có 2 lớp)
    y_prob = y_proba[:, 1] if y_proba.shape[1] > 1 else y_proba[:, 0]
   
    # 5. Đánh giá cơ bản
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print("\nKết quả đánh giá mô hình:")
    print(f"Accuracy:  {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall:    {recall * 100:.2f}%")
    print(f"F1-score:  {f1 * 100:.2f}%")

    # 6. ROC-AUC & ROC Curve, kể cả khi y_test chỉ có 1 lớp
    unique_classes = np.unique(y_test)
    if len(unique_classes) == 1:
        print(f"y_test chỉ chứa một lớp duy nhất ({unique_classes[0]}). Thêm điểm giả để tính ROC-AUC.")
        # Tạo bản sao an toàn
        y_test_sim = np.append(y_test.values, 1 - unique_classes[0])
        y_prob_sim = np.append(y_prob, 0.5)

        # Tính ROC
        roc_auc = roc_auc_score(y_test_sim, y_prob_sim)
        fpr, tpr, _ = roc_curve(y_test_sim, y_prob_sim)
    else:
        roc_auc = roc_auc_score(y_test, y_prob)
        fpr, tpr, _ = roc_curve(y_test, y_prob)

    print(f"ROC-AUC:   {roc_auc * 100:.2f}%")

    # Vẽ ROC
    plt.figure()
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

    # 7. Vẽ biểu đồ Feature Importances
    importances = model.feature_importances_
    top_indices = np.argsort(importances)[::-1][:20]
    top_features = X.columns[top_indices]
    top_values = importances[top_indices]

    plt.figure(figsize=(12, 6))
    plt.title("Top 20 Feature Importances")
    plt.bar(range(len(top_features)), top_values, align='center')
    plt.xticks(range(len(top_features)), top_features, rotation=90)
    plt.tight_layout()
    plt.show()

    return model

### VI. Run Test 

In [117]:
# =============================================================================
# Function: run_tests
# =============================================================================
def run_tests():
    """
    Chạy các kiểm thử cơ bản để xác nhận hàm preprocess_data hoạt động như mong đợi.
    """
    # Tạo một DataFrame mẫu với cấu trúc tương tự tập dữ liệu KDD Cup
    test_data = {
        'duration': [0, 0],
        'protocol_type': ['tcp', 'udp'],
        'service': ['http', 'domain_u'],
        'flag': ['SF', 'S0'],
        'src_bytes': [181, 239],
        'dst_bytes': [5450, 486],
        'land': [0, 0],
        'wrong_fragment': [0, 0],
        'urgent': [0, 0],
        'hot': [0, 0],
        'num_failed_logins': [0, 0],
        'logged_in': [1, 0],
        'num_compromised': [0, 0],
        'root_shell': [0, 0],
        'su_attempted': [0, 0],
        'num_root': [0, 0],
        'num_file_creations': [0, 0],
        'num_shells': [0, 0],
        'num_access_files': [0, 0],
        'num_outbound_cmds': [0, 0],
        'is_host_login': [0, 0],
        'is_guest_login': [0, 0],
        'count': [2, 3],
        'srv_count': [2, 3],
        'serror_rate': [0.0, 0.0],
        'srv_serror_rate': [0.0, 0.0],
        'rerror_rate': [0.0, 0.0],
        'srv_rerror_rate': [0.0, 0.0],
        'same_srv_rate': [1.0, 1.0],
        'diff_srv_rate': [0.0, 0.0],
        'srv_diff_host_rate': [0.0, 0.0],
        'dst_host_count': [150, 200],
        'dst_host_srv_count': [25, 30],
        'dst_host_same_srv_rate': [0.9, 0.8],
        'dst_host_diff_srv_rate': [0.1, 0.2],
        'dst_host_same_src_port_rate': [0.5, 0.5],
        'dst_host_srv_diff_host_rate': [0.0, 0.0],
        'dst_host_serror_rate': [0.0, 0.0],
        'dst_host_srv_serror_rate': [0.0, 0.0],
        'dst_host_rerror_rate': [0.0, 0.0],
        'dst_host_srv_rerror_rate': [0.0, 0.0],
        'label': ['normal.', 'smurf.']
    }
    test_df = pd.DataFrame(test_data)
    
    # Kiểm thử hàm preprocess_data
    X_test, y_test, df_cleaned_test = preprocess_data(test_df)
    # Kiểm tra xem cột 'label' có tồn tại trong dataset đã làm sạch hay không
    assert 'label' in df_cleaned_test.columns, "Error: Cleaned dataset must include the target variable."
    print("Preprocessing test passed.")

### VII. Main Function to run Complete pipeline

In [10]:
# =============================================================================
# Main function to run the complete pipeline
# =============================================================================
def main():
    """
    Hàm main thực thi toàn bộ quy trình data mining:
      1. Giải nén tập dữ liệu.
      2. Load dữ liệu.
      3. Tiền xử lý dữ liệu.
      4. Vẽ biểu đồ Confusion Matrix và Classification Report.    
      5. Chạy kiểm thử đơn vị.
      6. Huấn luyện và đánh giá mô hình.
    """
    filepath = '../../Data/kddcup.data_10_percent.csv'
      
    # Load dữ liệu
    data = load_data(filepath)
    if data is None:
        print("Dataset could not be loaded. Exiting.")
        return
    print("Dataset loaded successfully. Shape:", data.shape)
    
    # Tiền xử lý dữ liệu và lưu phiên bản đã làm sạch
    X, y, _ = preprocess_data(data)
    print("Data preprocessing completed.")

    # print(preprocess_data(data))
    
    # Chạy kiểm thử đơn vị
    run_tests()
    
    # Huấn luyện mô hình và đánh giá
    _ = train_and_evaluate(X, y)

    

# =============================================================================
# Run the main function when the script is executed
# =============================================================================
if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: '../Final_Project/Original_Data/Datakddcup.names'